In [104]:
#the xlrd library is necessary for read xls file  
#%pip install xlrd 

import pandas as pd
import numpy as np
import regex as re
from datetime import datetime
from time import time
from time import sleep
from math import ceil
from datetime import timedelta
import unicodedata  
import os
from IPython.display import clear_output
from difflib import SequenceMatcher 

In [105]:
%cd Data_Source

start_time = time()
#documentating information about the excel files and sheets inside of it.

#enter in the Data_Source folder to read the files
#listing the files on the directory
files = os.listdir()

#getting all the files with xlsx or xls format
xlsx_files = [x for x in files if x.split('.')[-1]=='xlsx' or  x.split('.')[-1]=='xls']

def get_sheets_names(x):
    xlsx = pd.ExcelFile(x)
    return xlsx.sheet_names  # see all sheet names

#putting all the sheets names inside a list
sheets_names = [get_sheets_names(excel) for excel in xlsx_files]

#zipping the lists togueter for execute as program
lst_src = list(zip(xlsx_files, sheets_names))

#building a dataframe with information about the excel files
df_xlsx = pd.DataFrame(lst_src, columns=['xlsx_file','sheets'])

for source in lst_src: print(f'EXCEL: {source[0]} >>> SHEETS: {source[1]}')

%cd ..

[Errno 2] No such file or directory: 'Data_Source'
/home/roberto/github/projeto_controle_financeiro_gesac-10/Data_Source
EXCEL: Controle de Empenhos e NC 2022.xlsx >>> SHEETS: ['ControleGeral', 'SintéticoGeral', 'SintéticoNC', 'AnalíticoNC', 'ÍndiceCorreçãoNC']
EXCEL: Controle de NC 2022.xlsx >>> SHEETS: ['SintéticoNC', 'AnalíticoNC', 'ÍndiceCorreção']
EXCEL: Emendas2021_resumo.xlsx >>> SHEETS: ['Planilha1']
EXCEL: Orçamentário MCOM 2021 16-11-21 setel.xlsx >>> SHEETS: ['Adm. Direta - Discricionárias', 'Emendas Individuais', 'Emendas de Bancada', 'dinamica', 'base']
EXCEL: Proponentes.xlsx >>> SHEETS: ['Sheet1']
EXCEL: Relatorio_SISGESAC 1-02-2022.xls >>> SHEETS: ['Extração']
EXCEL: estados.xlsx >>> SHEETS: ['estados']
/home/roberto/github/projeto_controle_financeiro_gesac-10


In [108]:
%cd Data_Source

#!!!
df_nm =[
('df_an', 'AnalíticoNC'),
# ('df_ei', 'Emendas Individuais'),
('df_sn', 'SintéticoNC'), 
('df_ic', 'ÍndiceCorreção'),
('df_pr', 'Sheet1'),
('df_cg', 'ControleGeral'),
('df_em', 'Planilha1'),
('df_estados', 'estados'),
('df_sg', 'Extração')] 

df_sheets = pd.DataFrame(df_nm, columns = ['df_name', 'sheet_name'])
df_sheets['xlsx_file']= ''

for i1, sheets in enumerate(df_xlsx['sheets']):
    for i2, sheet in enumerate(df_sheets['sheet_name']):
        if sheet in sheets:
            df_sheets['xlsx_file'][i2] = df_xlsx['xlsx_file'][i1] 

def parameters(df):
    excel = df_sheets.loc[df_sheets.df_name == df]['xlsx_file'].values[0]
    sheet = df_sheets.loc[df_sheets.df_name == df]['sheet_name'].values[0]
    return excel, sheet

xlsx, sheet = parameters('df_an')
df_an = pd.read_excel(xlsx,sheet_name=sheet,
usecols="B:J", skiprows=1,parse_dates=True)

xlsx, sheet = parameters('df_sn')
df_sn = pd.read_excel(xlsx, sheet_name=sheet,
usecols="B:J", skiprows=3,parse_dates=True)

xlsx, sheet = parameters('df_ic')
df_ic = pd.read_excel(xlsx, sheet_name=sheet,
skiprows=1,usecols="B:E")

xlsx, sheet = parameters('df_pr')
df_pr = pd.read_excel(xlsx)

xlsx, sheet = parameters('df_cg')
df_cg = pd.read_excel(xlsx, sheet_name=sheet
, skiprows=12,usecols="c:w")

xlsx, sheet = parameters('df_em')
df_em= pd.read_excel(xlsx, sheet_name=sheet)

# xlsx, sheet = parameters('df_ei')
# df_ei = pd.read_excel(xlsx)

xlsx, sheet = parameters('df_estados')
df_estados = pd.read_excel(xlsx)

xlsx, sheet = parameters('df_sg')
df_sg= pd.read_excel(xlsx)

#In general the excel files with just one sheet don't have a good name, so the name was changed to be similar to the name of the excel file
for i, sheet in enumerate(df_xlsx['sheets']):
    if len(sheet)==1:
        new_name = df_xlsx['xlsx_file'][i]
        re_pattern = '(.xlsx?|\d|-|\s)'
        new_name = re.sub(re_pattern,'',new_name)
        df_sheets.sheet_name.loc[df_sheets.sheet_name == sheet[0]] = new_name
    else: pass

for index_1, xlsx_1 in enumerate(df_xlsx['xlsx_file']):
    for index_2, xlsx_2 in enumerate(df_sheets['xlsx_file']):
        if xlsx_1 == xlsx_2 and len(df_xlsx['sheets'][index_1])==1:
            df_xlsx['sheets'][index_1] = df_sheets['sheet_name'][index_2]
        else: pass

#----------------------------------------------------------
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time

#Getting out of the Data_Source folder after having read the files
%cd Data_Source

/home/roberto/github/projeto_controle_financeiro_gesac-10/Data_Source
Time: This code took 48 seconds to run.
[Errno 2] No such file or directory: 'Data_Source'
/home/roberto/github/projeto_controle_financeiro_gesac-10/Data_Source


In [109]:
#Getting out of the Data_Source folder after having read the files
%cd ..

/home/roberto/github/projeto_controle_financeiro_gesac-10


In [110]:
start_time = time()

def strip_accents(cln):
   cln = str(cln)
   return ''.join(c for c in unicodedata.normalize('NFD', cln)if unicodedata.category(c) != 'Mn')

def clean_column_name(cln):
    return strip_accents(str(cln).strip('[]()').lower().replace(' ','_'))

def rename_df(df):
    new_cln = [clean_column_name(cln) for cln in df.columns.values]
    old_cln = [cln for cln in df.columns.values]
    dct = dict(zip(old_cln, new_cln))
    df.rename(columns=dct,inplace=True)
    lst_df_clns.append(new_cln)
    return df

#creating a list with all dfs from the data dictionary
dfs = [ x for x in df_sheets.df_name]
cmd = str(dfs).replace("'","") 
exec('dfs = {}'.format(cmd))

lst_df_clns = []
for df in dfs:
    rename_df(df)


df_sheets['columns'] = lst_df_clns 

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time

Time: This code took 1 seconds to run.


In [111]:
df_xlsx

,xlsx_file,sheets
0,Controle de Empenhos e NC 2022.xlsx,"[ControleGeral, SintéticoGeral, SintéticoNC, A..."
1,Controle de NC 2022.xlsx,"[SintéticoNC, AnalíticoNC, ÍndiceCorreção]"
2,Emendas2021_resumo.xlsx,Emendas_resumo
3,Orçamentário MCOM 2021 16-11-21 setel.xlsx,"[Adm. Direta - Discricionárias, Emendas Indivi..."
4,Proponentes.xlsx,Proponentes
5,Relatorio_SISGESAC 1-02-2022.xls,Relatorio_SISGESAC
6,estados.xlsx,estados


In [112]:
df_sheets

,df_name,sheet_name,xlsx_file,columns
0,df_an,AnalíticoNC,Controle de NC 2022.xlsx,"[proponente, nota_de_credito, programa, mes, v..."
1,df_sn,SintéticoNC,Controle de NC 2022.xlsx,"[nc, proponente, programa, empenho, valor_inic..."
2,df_ic,ÍndiceCorreção,Controle de NC 2022.xlsx,"[mes, indice_de_correcao, taxa_selic, observacao]"
3,df_pr,Proponentes,Proponentes.xlsx,"[unnamed:_0, prop_pk1, uf, partido, orgao, pro..."
4,df_cg,ControleGeral,Controle de Empenhos e NC 2022.xlsx,"[processo, proponente, contrato, nota_de_empen..."
5,df_em,Emendas_resumo,Emendas2021_resumo.xlsx,"[processo, emenda, valor, pontos_livre_(18_mes..."
6,df_estados,estados,estados.xlsx,"[cod, nome, sigla]"
7,df_sg,Relatorio_SISGESAC,Relatorio_SISGESAC 1-02-2022.xls,"[cdgesac, nrpid, situacao, sgunidadefederacao,..."


In [113]:
# criando função 'regexing' p/ subst. nomes abreviados de forma a normalizar as strings
lst_re = [
('PRF ','POLICIA RODOVIARIA FEDERAL '),
('MCOM ', 'MINISTERIO DAS COMUNICACOES '),
('BANCADA ','BANCADA-'),
(r'(DA BANCADA-?|INDIVIDUAL)',''),
(r'-D\w ','-'),
(r'\s(PARA|AJUDAR|DURANTE|A PANDEMIA DE COVID-19|DE\.)\s',''),
(r'\s(PROJETO|DA BANCADA|WIFI-FI NA|PRACA|ALTERANDO)\s',''),
(r'^PONTO\s(.*)DIA$',''),
(r'(\n)',''),
(r'SEN\.?(DOR\w?)?',''),
(r'(\s?DEPA?\.|\s?DEPUTAD\w?)\s?\s?(FEDERAL)?\s?',''),
(r'MIN/.?\s','MINISTERIO '),
(r'SOLICITA(CAO|d\w|N?DO)\s?(D\w|PEL\w|PARA\sO|PARA\sA)?',''),
(r'\w?\s?PEDIDO\s(FEIT\w|D\w)?\s?(PEL\w)?',''),
(r'-\s\w*',''),
(r'\(.*\)',''),
]

uf_re = zip(df_estados['nome'], df_estados['sigla'])
for x in list(uf_re):
    lst_re.append(x)

In [114]:
start_time = time()

#criando lista dos DataFrames pelo qual o loop passara referente aos proponentes
dfs = (df_sn, df_an,df_cg, df_em, df_sg)

#criando as colunas das primary_keys nos 3 dataframes
for df in dfs:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''

def regexing(value):
   for string in lst_re:
        value = re.sub(string[0],string[1],value)
   return value

pk2 = []
for index, prop in enumerate(df_pr['prop_pk1']):
    if df_pr['orgao'][index] == "senado":
        pk2.append('SEN. ' + prop)
    elif  df_pr['orgao'][index] == "camara":
        pk2.append('DEP. ' + prop)
    else: 
        pk2.append(prop)
df_pr['prop_pk2'] = pk2

#criando lista dos DataFrames pelo qual o loop passara referente aos proponentes
dfs = (df_sn, df_an,df_cg, df_em, df_sg)

#criando as colunas das primary_keys nos 3 dataframes
for df in dfs:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
#Time: This ckode took 1 seconds to run.

Time: This code took 1 seconds to run.


In [115]:
start_time = time()

df_sg['proponente'] = df_sg['nosolicitante'] 
df_em['proponente'] = df_em['emenda']

#fazendo o loop sobre a lista acima para aplicar as funções que normalizam a coluna com as chaves dos proponentes
lst_df = [df_sn, df_an, df_cg, df_em, df_sg]

for df in lst_df: 
    df['proponente'] = df['proponente'].apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
    df['proponente'] = df['proponente'].apply(lambda x:str(x).strip())
  
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 4 seconds to run.


O objetivo inicial do projeto foi limpar os dados relacionados ao os proponentes com a criação de uma coluna nova referente ao nome civil do proponente para que esses valores pudessem ser usados como chave entre diferentes tabelas que serao parametrizadas entre si.

In [116]:
start_time = time()

lst_pr = (df_pr['prop_pk1'], df_pr['prop_1'],df_pr['prop_2'], df_pr['prop_3'],df_pr['prop_4'])

for cln in lst_pr:
    cln = cln.apply(lambda x:strip_accents(x).upper())

#criando lista das colunas referente ao nome dos proponentes 
def filter_key(x):
    return (x==lst_pr[0])|(x==lst_pr[1])|(x==lst_pr[2])|(x==lst_pr[3])|(x==lst_pr[4])

for df in lst_df:
    lst_prop_pk1 = []
    lst_prop_pk2 = []
    for index,value in enumerate(df.proponente):
        filt_finding_key =  filter_key(df.proponente.iloc[index])
        result1 = str(df_pr.loc[filt_finding_key].prop_pk1.values).strip("[']")
        result2 = str(df_pr.loc[filt_finding_key].prop_pk2.values).strip("[']")
        lst_prop_pk1.append(result1)
        lst_prop_pk2.append(result2)
    df['prop_pk1'] = lst_prop_pk1
    df['prop_pk2'] = lst_prop_pk2

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 35 seconds to run.


In [117]:
start_time = time()
#cleaning analitic sintetic rows

lst = ['SUBTOTAL','TOTAL','NAN']

for value in lst:
    df_sn = df_sn.loc[df_sn['proponente'] != value ]  
    df_em = df_em.loc[df_em['proponente'] != value ]  
    
df_sn.fillna(0,inplace=True)
df_an.fillna(0,inplace=True)

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 1 seconds to run.


In [119]:
#df_em['bancada']=''
#df_em['individual']= ''
#df_em.individual = df_em.emenda.apply(lambda x: False if 'individual' not in x else True) 
#df_em.bancada = df_em.emenda.apply(lambda x: str(x).split('bancada ')[-1].strip()[:-1] if '(bancada' in x else '') 

In [120]:
lst_df_em_proponentes = []
for prop in df_em['proponente'].loc[df_em['prop_pk1']=='']:
    if prop not in lst_df_em_proponentes and prop!= '':
        lst_df_em_proponentes.append(prop)

props_pks = []
for prop in lst_df_em_proponentes:
    for prop_pk in df_pr['prop_pk1'].astype('str'):
        if prop in prop_pk or prop_pk in prop:
            props_pks.append((prop,prop_pk))
        else:pass

props_pks

[('FREDERICO', 'FREDERICO DE CASTRO ESCALEIRA'),
 ('FREDERICO', 'FREDERICO BORGES DA COSTA')]

In [121]:
key = [pks for props, pks in props_pks]
proponente = [props for props, pks in props_pks]

program = 0 #0=off; 1=on
save = 0 
if program == True:
    for index, value in enumerate(zip(key,proponente)):
    
        clear_output(wait=True);sleep(1)
        for i in range(4,0,-1): 
            exec(f"bool_v{i} = str(df_pr.loc[df_pr.prop_pk1 == key[{index}]].prop_{i}.values).strip('[]') == 'nan'")
            exec(f'lst_bool.append(bool_v{i})')
            exec(f'if bool_v{i} == True: number = {i}')
        
        #menu_program
        print(f'PROPONENTE: {proponente[index]}\nPK: {key[index]}')
        over_view = df_pr.loc[df_pr.prop_pk1 == key[index]][['prop_1','prop_2','prop_3','prop_4']]
        print(over_view)
        print('EXECUTE ?'); rule = int(input('EXECUTE ?'))
        
        if rule == True:
            exec(f"df_pr.loc[df_pr.prop_pk1 =='{key[index]}', 'prop_{number}'] = '{proponente[index]}'")
else: 
    pass

if save == True:
    %cd Data_Source
    df_pr.to_excel('Proponentes.xlsx')
    %cd ..

In [122]:
#searching for proponetes in df_pr by the prop_n
name = 'SERGIO PETECAO'
#criando lista das colunas referente ao nome dos proponentes 
lst_cl = ['prop_pk1', 'prop_1', 'prop_2', 'prop_3', 'prop_4']
delete = 0

for i in range(4,0,-1): 
    column = 'prop_' + str(i)
    value = df_pr.loc[df_pr[column] == name][column].values
    if value.size > 0:
        my_output = df_pr.loc[df_pr[column] == name][lst_cl]
        if delete == True:
            df_pr[column] = df_pr[column].replace(name,np.nan)
            break
        break

my_output

,prop_pk1,prop_1,prop_2,prop_3,prop_4
101,SERGIO PETECAO,SERGIO PETECAO,NaN,NaN,NaN


In [123]:
start_time = time()
dfs = [df_an, df_ic]

for index,df in enumerate(dfs): 
    meses = [mes for mes in df.mes.unique()]
    n_mes = ['01/2021','02/2021','03/2021','04/2021','05/2021','06/2021','07/2021','08/2021','09/2021','10/2021','11/2021','12/2021']
    meses = list(zip(meses, n_mes))

    for index,value in enumerate(meses):
        df['mes'].replace(meses[index][0],meses[index][1],inplace=True)


    df['mes']= pd.to_datetime(df['mes'])
    if (df.mes.dtype == '<M8[ns]') == True:
        print(f'The loop is ok!')
        
    else: print('The loops is broke!')
    
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

The loop is ok!
The loop is ok!
Time: This code took 1 seconds to run.


In [124]:
#checando se tem dfs sem prop_pk 
start_time = time()

dfs  = [df_an, df_sn,df_em, df_sg]
lst_no_cv_name = []
for df in dfs:
    for index, value in enumerate(df.loc[df.prop_pk1==''].proponente.values):
        if value not in lst_no_cv_name:
            lst_no_cv_name.append(df.loc[df.prop_pk1==''].proponente.values[index])
        else:pass

print('DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:')
df_nk = pd.DataFrame()
df_nk['proponente']= lst_no_cv_name

save = True
if save == True:
    %mkdir trash
    df_nk.to_excel('trash/props_with_no_key.xlsx')
else:pass

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:
mkdir: cannot create directory ‘trash’: File exists
Time: This code took 1 seconds to run.


In [125]:
lst_pr = (df_pr['prop_pk1'], df_pr['prop_1'],df_pr['prop_2'], df_pr['prop_3'],df_pr['prop_4'])

def filter_try(x):
    return (x==lst_pr[0])|(x==lst_pr[1])|(x==lst_pr[2])|(x==lst_pr[3])|(x==lst_pr[4])

lst_pk1 = []
lst_pk2 = []

for index,value in enumerate(df_nk):
    filt_finding_key =  filter_try(df_nk.proponente.iloc[index])
    result1 = str(df_pr.loc[filt_finding_key].prop_pk1.values).strip("[']")
    result2 = str(df_pr.loc[filt_finding_key].prop_pk2.values).strip("[']")
    print(result1)
    print(result2)

print(lst_pk1)



[]


In [97]:
# seq=difflib.SequenceMatcher(None, a,b)
# d=seq.ratio()*100

#lst_match = []
#for pr_nk in df_nk.proponente:
#    for pr_pk in df_pr.prop_pk1:
#        result = SequenceMatcher(None,pr_nk,pr_pk)
#        lst_match.append((pr_nk)result)
#    print



In [126]:
#df_folders
lst_fd = [
    ('FINANCEIRO','\GESAC\GESAC_2017\ADMPP'),
    ('SISGESAC',None)
    ]

fd = [fd for (pj,fd) in lst_fd]
pj = [pj for (pj,fd) in lst_fd]
df_folders = pd.DataFrame({'project':pj,'folder':fd})

In [127]:
%rm -r Data_Dictionary
%mkdir Data_Dictionary
%mkdir Data_Dictionary/csv/
%mkdir Data_Dictionary/xlsx/
%mkdir Data_Dictionary/csv/Columns/
%mkdir Data_Dictionary/xlsx/Columns/

path = 'Data_Dictionary'

for df in ('df_sheets', 'df_xlsx', 'df_folders'):
    exec(f"{df}.to_csv('{path}/csv/{df}.csv')")
    exec(f"{df}.to_excel('{path}/xlsx/{df}.xlsx')")


path2 = 'Columns'

for i, df  in enumerate(df_sheets['df_name']):
       exec(f'df = {df}')
       lst_cl = []
       for cl in df.columns: lst_cl.append(cl)
       dct = {'columns':lst_cl}
       exec(f"{df_sheets['df_name'][i]}_columns = pd.DataFrame(dct)")
       exec(f"{df_sheets['df_name'][i]}_columns['descricao'] = '' ")
       exec(f"{df_sheets['df_name'][i]}_columns.to_csv('{path}/csv/{path2}/{df_sheets['sheet_name'][i]}_columns.csv')")
       exec(f"{df_sheets['df_name'][i]}_columns.to_excel('{path}/xlsx/{path2}/{df_sheets['sheet_name'][i]}_columns.xlsx')")

In [128]:
final_time= str(timedelta(seconds= total_time)); print(f'Time: The total time to run all the notebook was {final_time}.')

Time: The total time to run all the notebook was 0:00:37.812318.
